# Universidad de Monterrey

## División de Ingenierías

### Lab - Robotics

Lab 10: Camera calibration and image distortion correction

Authors: Miguel Benavides, Laura Morales



### Introduction
For the final robotics lab practice, we will investigate about image correction by calibrating the camera using the OpenCV library. To test this, we will take aproximately 15 to 20 pictures of a chessboard from different angles. The chessboard was used due to the straight lines that are easily distorted by the camera from the different positions and angles. By investigating about this, we should find some methods that will filter and clean all these images and create a final image with straight lines.


### Procedure
For this practice, as well as the previous ones, the following tools are required:

* Raspberry Pi with WiFi connection capabilities
* Jupyter Notebook
* Python >= 3.5.2
* OpenCV 3.2
* Git
* GitHub account
* Markdown editor (recommended: ReText 5.3.1)


### Camera calibration using OpenCV

Pinhole cameras may cause image distortion due to the lens'length. The two main distortions are: radial distortion and tangential distortion.

The following code recovers several images of a chess board taken from different angles, all of these images are affected by some kind of distortion, then with the use of OpenCV, images will be corrected in order to retrieve a final image with all the corrected borders.

This code detects the image borders and adds points in each of the squares in order to compare the distances and straighten the curved lines. 


In [ ]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

img = cv2.imread('2018-05-01-131901.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)
cv2.imshow('Calibration Result',dst)

tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

mean_error = tot_error/len(objpoints)
print("total error: ", mean_error)

cv2.waitKey()

### Conclusions

Miguel: With this lab I learned more about the camera library, more specifically, more about camera calibration functions, like findchessboardcorners() to fix points on an image and then correlate the position of the points with the curvature caused by deformation. Then, with this points we used the calibratecamera() function, in order to deform the image in a way that the pre-curved lines between the points were now straight, this way the whole image is corrected from the deformation caused by the camera lenses, and we can therefore apply this calibration values to the camera for any photo taken afterwards.

Laura: This final practice gave me a new perspective of how complex and advanced are the cameras that we use everyday. I specially found the class explanation about the focal point and how the distance between it and the sensor affects the image captured to be very interesting. I think it is amazing how much technology we take for granted due to the fact that we use it everyday but is far more complex than we think. This lab helped me understand how cameras process images and apply this corrections to give us better results.